In [1]:
from collections import namedtuple
import operator
from functools import *
from itertools import *
import math

In [2]:
## Ćwiczenie 1.

# No lambda usage
def pairs_to_triples(pairs_iterator):
    pairs1, pairs2 = tee(pairs_iterator)
    
    powers = starmap(pow, pairs1)
    
    return starmap(chain, zip(pairs2, map(repeat, powers, repeat(1))))

# Test:
pairs = [(2, 3), (4, 5), (5, 6)]
result = pairs_to_triples(iter(pairs))
print("Results for no lambda:")
for triple in result:
    print(tuple(triple))

# Only lambda
pairs_to_triples_lambda = lambda it: (lambda p1, p2: starmap(lambda p, r: chain(p, r), zip(p2, map(repeat, starmap(pow, p1), repeat(1)))))(*tee(it))

# Test:
result = pairs_to_triples_lambda(iter(pairs))
print("Results for lambda:")
for triple in result:
    print(tuple(triple))

Results for no lambda:
(2, 3, 8)
(4, 5, 1024)
(5, 6, 15625)
Results for lambda:
(2, 3, 8)
(4, 5, 1024)
(5, 6, 15625)


In [3]:
## Ćwiczenie 2.

# Create a tree
def zrob_drzewo(n, iterable):
    if n <= 0:
        return None
    
    elements = list(islice(iterable, n))

    left_size = (n - 1) // 2
    right_size = n - 1 - left_size

    root = elements[left_size]
    left = zrob_drzewo(left_size, elements[:left_size])
    right = zrob_drzewo(right_size, elements[left_size + 1:])
    
    return (left, root, right)


# Print tree using yield
def obejdź_yield(drzewo, preorder=False, inorder=False, postorder=False):

    if sum([preorder, inorder, postorder]) != 1:
        raise ValueError("More then one true value!")
        
    left, root, right = drzewo
    
    if preorder:
        yield root
    
    if left is not None:
        yield from obejdź_yield(left, preorder, inorder, postorder)
    
    if inorder:
        yield root
        
    if right is not None:
        yield from obejdź_yield(right, preorder, inorder, postorder)
        
    if postorder:
        yield root

# Print tree using chain
def obejdź_chain(drzewo, preorder=False, inorder=False, postorder=False):

    if sum([preorder, inorder, postorder]) != 1:
        raise ValueError("More then one true value!")
        
    left, root, right = drzewo
    
    left_iter = obejdź_chain(left, preorder, inorder, postorder) if left is not None else iter(())
    right_iter = obejdź_chain(right, preorder, inorder, postorder) if right is not None else iter(())
    root_iter = iter([root])
    
    if preorder:
        return chain(root_iter, left_iter, right_iter)
    elif inorder:
        return chain(left_iter, root_iter, right_iter)
    else:
        return chain(left_iter, right_iter, root_iter)

# Tests
# Test tree creation
tree = zrob_drzewo(7, "alakota")
print("Created tree:", tree)
    
# Test both versions of prints
print("\nYield version:")
print("Inorder:", list(obejdź_yield(tree, inorder=True)))
print("Preorder:", list(obejdź_yield(tree, preorder=True)))
print("Postorder:", list(obejdź_yield(tree, postorder=True)))
    
print("\nChain version:")
print("Inorder:", list(obejdź_chain(tree, inorder=True)))
print("Preorder:", list(obejdź_chain(tree, preorder=True)))
print("Postorder:", list(obejdź_chain(tree, postorder=True)))

Created tree: (((None, 'a', None), 'l', (None, 'a', None)), 'k', ((None, 'o', None), 't', (None, 'a', None)))

Yield version:
Inorder: ['a', 'l', 'a', 'k', 'o', 't', 'a']
Preorder: ['k', 'l', 'a', 'a', 't', 'o', 'a']
Postorder: ['a', 'a', 'l', 'o', 'a', 't', 'k']

Chain version:
Inorder: ['a', 'l', 'a', 'k', 'o', 't', 'a']
Preorder: ['k', 'l', 'a', 'a', 't', 'o', 'a']
Postorder: ['a', 'a', 'l', 'o', 'a', 't', 'k']


In [5]:
## Ćwiczenie 3.
def odpluskw(f):
    def debug_f(*args, **kwargs):
        print(f"\nCalling {f.__name__}")
        if args:
            print("Args:")
            for i, arg in enumerate(args):
                print(f"  arg{i}: {arg}")
            for key, value in kwargs.items():
                print(f"  {key}: {value}")
            
        result = f(*args, **kwargs)
        print(f"Result: {result}")
        return result
    return debug_f
    
def power_tower(listed):
    return reduce(lambda x, y: y**x, reversed(listed))

def count_digits(n):
    if n == 0:
        return 1
    return math.floor(math.log10(abs(n))) + 1

@odpluskw
def power_tower_debug(listed):
    return reduce(lambda x, y: y**x, reversed(listed))

@odpluskw
def count_digits_debug(n):
    if n == 0:
        return 1
    return math.floor(math.log10(abs(n))) + 1

@odpluskw
def add_towers_digits_debug(list_1, list_2):
    return count_digits(power_tower(list_1)+power_tower(list_2))

# Test
power_tower_debug([2, 3, 4])

count_digits_debug(355**10+340**11)
count_digits_debug(count_digits(10**10+340**5))

add_towers_digits_debug([3, 4, 5], [4, 5, 6])



Calling power_tower_debug
Args:
  arg0: [2, 3, 4]
Result: 2417851639229258349412352

Calling count_digits_debug
Args:
  arg0: 7050673851606069271103515625
Result: 28

Calling count_digits_debug
Args:
  arg0: 13
Result: 2

Calling add_towers_digits_debug
Args:
  arg0: [3, 4, 5]
  arg1: [4, 5, 6]
Result: 9408


9408